<a href="https://colab.research.google.com/github/shenzhun/machine-learning-prep/blob/master/tensorflow/day15_losses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, losses, regularizers, constraints

In [10]:
tf.keras.backend.clear_session()
model = models.Sequential()
model.add(
    layers.Dense(64, 
                 input_dim=64,
                 kernel_regularizer=regularizers.l2(0.01),
                 activity_regularizer=regularizers.l1(0.01),
                 kernel_constraint=constraints.MaxNorm(max_value=2, axis=0)
    )
)
model.add(
    layers.Dense(10,
                 kernel_regularizer=regularizers.l1_l2(0.01, 0.01),
                 activation='sigmoid')
    )
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['AUC']
              )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 4,810
Trainable params: 4,810
Non-trainable params: 0
_________________________________________________________________


In [11]:
def focal_loss(gamma=2., alpha=0.75):
  def focal_loss_fixed(y_true, y_pred):
    bce = tf.losses.binary_crossentropy(y_true, y_pred)
    p_t = (y_true * y_pred) + ((1 - y_true) * (1 - y_pred))
    alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
    modulating_factor = tf.pow(1.0 - p_t, gamma)
    loss = tf.reduce_sum(alpha_factor * modulating_factor * bce, axis=-1)
    return loss
  return focal_loss_fixed

In [12]:
class FocalLoss(losses.Loss):
  def __init__(self, gamma=2.0, alpha=0.75, name="focal_loss"):
    self.gamma = gamma
    self.alpha = alpha
  
  def call(self, y_true, y_pred):
    bce = tf.losses.binary_crossentropy(y_true, y_pred)
    p_t = (y_true * y_pred) + ((1 - y_true) * (1 - y_pred))
    alpha_factor = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
    modulating_factor = tf.pow(1.0 - p_t, self.gamma)
    loss = tf.reduce_sum(alpha_factor * modulating_factor * bce, axis=-1)
    return loss